In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [26]:
gc = GraphCreator("Sonata form", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links))
print(list(gc.primary_nodes.keys()))

Number of Links to Search: 1023
['Symphony', 'Recapitulation (music)', 'Tonality', 'Musical form', 'Coda (music)', 'Movement (music)', 'Musical analysis', 'Musical development', 'String quartet', 'Exposition (music)', 'Introduction (music)', 'Sonata', 'Classical music era', 'Concerto']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [27]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [28]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [29]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Ludwig van Beethoven,Wolfgang Amadeus Mozart,22
1,Camille Saint-Saëns,Charles-Valentin Alkan,22
2,Wolfgang Amadeus Mozart,Ludwig van Beethoven,22
3,Charles-Valentin Alkan,Camille Saint-Saëns,22
4,Camille Saint-Saëns,Claude Debussy,21


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [30]:
features_df = gc.get_features_df(rank=False)

In [31]:
features_df.sort_values("degree", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,primary_link
0,Composer,15802,0,15296.0,506.0,0.005663,1.742891e-02,0.028116,230.0,2.0,1.0,0
1,Classical music,10900,0,9986.0,914.0,0.010536,2.210821e-02,0.021352,626.0,2.0,1.0,0
2,Opera,10221,0,9438.0,783.0,0.010628,2.459935e-02,0.018076,796.0,2.0,1.0,0
3,Baroque,8202,1,7672.0,530.0,0.003297,6.007730e-03,0.021045,406.0,2.0,2.0,0
4,Music,7820,0,6665.0,1155.0,0.013569,6.633977e-03,0.017565,562.0,2.0,1.0,0
5,Johann Sebastian Bach,5498,1,4627.0,871.0,0.027861,1.393236e-01,0.008673,992.0,2.0,2.0,0
6,19th century,5369,0,3718.0,1651.0,0.005726,5.393575e-04,0.007918,448.0,2.0,1.0,0
7,Orchestra,4933,0,4587.0,346.0,0.026277,1.837453e-02,0.008579,272.0,1.0,2.0,0
8,Wolfgang Amadeus Mozart,4930,0,4565.0,365.0,0.043642,2.346291e-01,0.007980,418.0,1.0,2.0,0
9,Ludwig van Beethoven,4889,1,4209.0,680.0,0.057878,3.280981e-01,0.007700,904.0,2.0,2.0,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [32]:
gc.rank_similarity()

# print(gc.see_also_articles)

gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shared_neighbors_with_entry_score",
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "adjusted_reciprocity",
    "centrality",
    "similarity_rank", 
]].sort_values(["similarity_rank"], ascending=False).reset_index().drop("index", axis=1).head(50)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

,node,category_matches_with_source,primary_link,shared_neighbors_with_entry_score,shortest_path_length_from_entry,shortest_path_length_to_entry,adjusted_reciprocity,centrality,similarity_rank
0,Sonata form,1,0,1.000000,0.0,0.0,372.0,0.030495,inf
1,Recapitulation (music),0,1,0.195701,1.0,1.0,88.0,0.005648,1.195701
2,Exposition (music),0,1,0.173611,1.0,1.0,88.0,0.004477,1.173611
3,Sonata,0,1,0.139249,1.0,1.0,236.0,0.011256,1.139249
4,Coda (music),0,1,0.137091,1.0,1.0,104.0,0.007964,1.137091
5,Movement (music),0,1,0.124577,1.0,1.0,86.0,0.008197,1.124577
6,Musical development,1,1,0.196210,2.0,2.0,92.0,0.006999,1.098105
7,Symphony,0,1,0.084288,1.0,1.0,136.0,0.036741,1.084288
8,Introduction (music),0,1,0.078597,1.0,1.0,88.0,0.001810,1.078597
9,String quartet,0,1,0.058415,1.0,1.0,712.0,0.016262,1.058415


# Evaluation

In [34]:
evaluate_metrics(gc.features_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.995402,0.999973,0.004571,3.0,1.000000,1.000000,1.0,1.0
centrality,0.939762,0.999973,0.060211,3.0,0.333333,0.666667,1.0,1.0
adjusted_reciprocity,0.944261,0.999973,0.055712,3.0,0.666667,0.666667,1.0,1.0
page_rank,0.968734,0.999973,0.031239,3.0,0.666667,1.000000,1.0,1.0
shortest_path_length_from_entry,0.715856,0.999973,0.284117,3.0,0.000000,0.000000,0.0,0.0
